In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
import math

from collections import Counter
#pathPrefix = '/Users/friedman/Desktop/mnt'
#sys.path.append(pathPrefix + '/ifs/work/taylorlab/friedman/myUtils')
#import analysis_utils 
#import mutationSigUtils

In [2]:
def summarize_n_hypermutated_and_signatures_of_cases(mutClassificationDir = '/Users/friedman/Desktop/hypermutationStatusIds/'):
    nCasesSummaryL = []
    signaturesSummaryL = []
    for f in os.listdir(mutClassificationDir):
        cancerType = re.sub('.tsv', '', f)
        filePath = os.path.join(mutClassificationDir, f)
        df = pd.read_table(filePath)
        hypermutatedDf = df[df['hypermutantClassification'] == 'Hypermutated']
        highMutBurdenDf = df[df['hypermutantClassification'] == 'highMutationBurden']
        
        nCasesSummaryL.append({'cancerType': cancerType,
        'nHypermutated': hypermutatedDf.shape[0], 'nTotal': df.shape[0], 'nHighMutBurden': highMutBurdenDf.shape[0]})
    
        #TODO make the code for plotting the signatures
        signatureCounter = Counter(hypermutatedDf['dominantSignature'])
        for signature, count in signatureCounter.items():
            signature = re.sub('mean_', '', signature)
            signaturesSummaryL.append({'cancerType': cancerType,
                'signature': signature, 'nCases': count, 
                'nHypermutatedCases': hypermutatedDf.shape[0], 'nTotal': df.shape[0]})
        
    casesSummaryDf = pd.DataFrame(nCasesSummaryL)
    signaturesSummaryDf = pd.DataFrame(signaturesSummaryL)
    return casesSummaryDf, signaturesSummaryDf
        

In [8]:
def summarize_dominant_signatures_of_cases(mutClassificationDir = '/Users/friedman/Desktop/hypermutationStatusIds/'):
    d = {}
    for f in os.listdir(mutClassificationDir):
        cancerType = re.sub('.tsv', '', f)
        filePath = os.path.join(mutClassificationDir, f)
        df = pd.read_table(filePath)
        hypermutatedDf = df[df['hypermutantClassification'] == 'Hypermutated']
        domSigDict = dict(zip(hypermutatedDf['Tumor_Sample_Barcode'], hypermutatedDf['dominantSignature']))
        for tsb, domSig in domSigDict.items():
            d[tsb] = domSig
    return d

In [3]:
casesSummary, signaturesSummary = summarize_n_hypermutated_and_signatures_of_cases()

In [9]:
dominantSignatureDict = summarize_dominant_signatures_of_cases(mutClassificationDir = '/Users/friedman/Desktop/hypermutationStatusIds/')

In [4]:
#ADJUST THE CASES SUMMARY INFORMATION
minNCasesToDisplay = 250 #lump all cancers with fewer than this number of cases as other
casesSummary['cancerType'] = casesSummary.apply(lambda row: 'other' if row['nTotal'] < minNCasesToDisplay else row['cancerType'] ,axis=1)


In [5]:
#ADJUST THE SIGNATURES SUMMARY INFORMATION
minNHypermutatedCasesToDisplay = 10
signaturesSummary['cancerType'] = signaturesSummary.apply(lambda row: 'other' if row['nHypermutatedCases'] < minNHypermutatedCasesToDisplay else row['cancerType'] ,axis=1)
signaturesSummary['orderingVal'] = signaturesSummary.apply(lambda row: -1 if row['cancerType'] == 'other' else row['nHypermutatedCases'], axis=1)

signaturesRenameDict = {'1': 'MMR', 'SMOKING': 'SMOKING',
                        'MMR': 'MMR', 'APOBEC': 'APOBEC', '10': 'POLE',
                       '11': 'TMZ', '14': 'POLE&MMR', '7': 'UV'}
signaturesSummary['signature'] = signaturesSummary['signature'].apply(lambda x:
        signaturesRenameDict[x] if x in signaturesRenameDict else 'other')

In [8]:
cancersOfIndeterminateOrigin = set(['Cancer_of_Unknown_Primary', 'Skin_Cancer,_Non-Melanoma'])  #remove cancers of indeterminate origin
signaturesSummary = signaturesSummary[~signaturesSummary['cancerType'].isin(cancersOfIndeterminateOrigin)]
casesSummary = casesSummary[~casesSummary['cancerType'].isin(cancersOfIndeterminateOrigin)]


In [9]:
signaturesSummary.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure1cSignatureSummary.tsv', index=False, sep='\t')
casesSummary.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/figure1bCancerTypeSummary.tsv', index=False, sep='\t')

In [ ]:
def get_per_case_nmut_mb_info(nmutDfPath = pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/nmutInfo_impact_filtered.tsv'):
    df = pd.read_table(nmutDfPath)
    return dict(zip(df['Tumor_Sample_Barcode'], df['Nmut_Mb']))

**add signature information**

**make it a long format df**

In [5]:
signaturesSummary

,cancerType,nCases,nHypermutatedCases,nTotal,signature
0,Appendiceal_Cancer,1,1,221,MMR
1,Bladder_Cancer,2,47,1314,SMOKING
2,Bladder_Cancer,1,47,1314,16
3,Bladder_Cancer,1,47,1314,14
4,Bladder_Cancer,2,47,1314,10
5,Bladder_Cancer,9,47,1314,MMR
6,Bladder_Cancer,1,47,1314,28
7,Bladder_Cancer,1,47,1314,5
8,Bladder_Cancer,4,47,1314,7
9,Bladder_Cancer,1,47,1314,30
